In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

Data Load

In [12]:
df_original = pd.read_excel('../data/raw/tweets_12-27Mayo.xlsx')  
df_post = df_original['POST']
print("El dataframe tiene {} filas.".format(df_post.shape[0]))

El dataframe tiene 769 filas.


Load a pretrained model and its corresponding tokenizer. The model is cardiffnlp/twitter-roberta-base-sentiment, which is a model trained for sentiment analysis.

In [13]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

Define the sentiment labels: Negative, Neutral, and Positive.

In [14]:
labels = ['Negative', 'Neutral', 'Positive']


The data frame df is defined to be df_post, which presumably contains the data you're interested in processing.

In [15]:
df = df_post
df.head()

0    Comenzamos la campaña, una vez más, junto a la...
1    Madrid es la región del Espíritu de Ermua, la ...
2    Majadahonda con ganas de Libertad, familia, un...
3                                                 ❤️❤️
4                        Presidente: líbranos del mal.
Name: POST, dtype: object

Define a function preprocess_tweet to preprocess tweets. This function replaces @mentions with "@user" and links (starting with http) with "http".

In [17]:
def preprocess_tweet(tweet):
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)
    return " ".join(tweet_words)


In [16]:
def preprocess_tweet(tweet):
    if isinstance(tweet, float):
        return ''
    
    tweet_words = []
    for word in str(tweet).split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http') or word.startswith('www'):
            word = '@url'
        tweet_words.append(word)
    
    return ' '.join(tweet_words)

Apply the preprocess_tweet function to all tweets in the dataframe and store the results in a new column named processed_text.

In [17]:
df['processed_text'] = df.apply(preprocess_tweet)

Define a function analyze_sentiment to analyze the sentiment of tweets using the loaded model and tokenizer. This function returns a dictionary of sentiment labels and their corresponding scores.

In [18]:
def analyze_sentiment(tweet):
    encoded_tweet = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return dict(zip(labels, scores))


Apply the analyze_sentiment function to all processed tweets in the dataframe and store the results in a new column named sentiment_analysis.

In [19]:
df['sentiment_analysis'] = df['processed_text'].apply(analyze_sentiment)


Break down the sentiment_analysis column into separate columns for each sentiment label.

In [20]:
sentiment_df = df['sentiment_analysis'].apply(pd.Series)


Concatenate the original data frame with the new sentiment dataframe along columns (axis=1).

In [21]:
df = pd.concat([df_original, sentiment_df], axis=1)


Determine the final sentiment for each tweet by choosing the sentiment label with the highest score, and store this in a new column named final_sentiment.

In [24]:
df['final_sentiment'] = sentiment_df.idxmax(axis=1)


We export it to csv.

In [26]:
df.to_csv('../data/processed/df_Sentiment_pred_roberta_base.csv')